## **05-3) 트리의 앙상블**

앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 알아본다.

### **앙상블 학습**(Ensemble Learning)

더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘으로, 정형 데이터를 다루는 데 가장 뛰어난 성과를 낸다.

<hr><br>

### **랜덤 포레스트**(Random Forest)

대표적인 결정 트리 기반의 앙상블 학습 방법으로, **부트스트랩 샘플**을 사용하고 전체 특성 중 일부를 랜덤하게 선택하여 결정 트리를 만드는 것이 특징이다.

**부트스트랩 샘플**<br>
부트스트랩 샘플은 부트스트랩 방식으로 샘플링하여 분류한 데이터를 의미하는데, **부트스트랩 방식**은 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식이다.예를 들어, 1,000개 가방에서 100개씩 샘플을 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣어 그다음 샘플을 뽑는다. 1,000개 가방에서 중복하여 1,000개의 샘플을 뽑기 때문에 부트스트랩 샘플은 훈련세트와 크기가 같다.

또한 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.<br>
분류모델인 RandomForestClaaifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택하고,<br>
회귀모델인 RandomForestRegressor는 전체 특성을 사용한다.

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련하며, <br>
분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다.<br>
회귀일 때는 단순히 각 트리의 예측을 평균한다.

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고<br>
검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

사이킷런의 `RandomForestClassifier` 클래스를 화이트 와인을 분류하는 문제에 적용해보자.

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

`return_train_score=True`로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환한다.<br> 훈련세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는 데 용이하다.

In [17]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


훈련 세트에 다소 과대적합되었다.

랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다.<br>
특성 중요도 역시 제공하는데, 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.<br>

랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력해보자.

In [18]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


이전의 `결정트리`에서 만든 특성 중요도와 비교했을 때<br>
두 번째 특성인 당도의 중요도가 감소하고, 알코올 도수와 pH 특성의 중요도가 조금 상승했다.<br>
랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.

RandomForestClassifier는 자체적으로 모델을 평가하는 점수를 얻을 수도 있다.<br>
부트스트랩 샘플을 만들 때 포함되지 않고 남는 샘플이 있는데, 이런 샘플을 **OOB**(Out Of Bag)**샘플**이라고 한다. 이 남는 샘플을 검증 세트처럼 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. 

In [19]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


`oob_score_`는 각 결정 트리의 OOB 점수를 평균하여 출력한다.<br>
교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻었다. OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

<hr><br>

### **엑스트라 트리**(Extra Tree)

랜덤 포레스트와 비슷하지만 부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용한다.<br>
또한, 노드를 분할할 때 최선이 아니라 랜덤하게 분할하기 때문에 랜덤 포레스트보다 훈련 속도가 빠르다. 그러나 보통 더 많은 트리가 필요하다.

`ExtraTreesClassifier` 모델의 교차 검증 점수를 확인해보자.

In [20]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻었다.

엑스트라 트리도 특성 중요도를 제공한다.

In [21]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


엑스트라 트리 역시 결정 트리보다 당도에 대한 의존성이 작은 것을 볼 수 있다.

<hr><br>

### **그레이디언트 부스팅**(Gradient Boosting)

랜덤 포레스트나 엑스트라 트리와 달리 깊이가 얕은 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법이다.<br>
성능이 뛰어나지만 병렬로 훈련할 수 없기 때문에 랜덤 포레스트나 엑스트라 트리보다 훈련 속도가 조금 느리다. 



사이킷런의 `GradientBoostingClassifier`를 사용하여 와인 데이터셋의 교차 검증 점수를 확인해보자.<br>
GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리 100개를 사용한다.

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대적합이 거의 되지 않은 것을 볼 수 있다.<br>
그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.

학습률(learning_rate, 기본값은 0.1)을 증가시키고 트리의 개수(n_estimators)를 늘리면 모델의 성능을 향상시킬 수 있다.

In [23]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 5배로 늘려도 과대적합을 잘 억제하고 있다.

그레이디언트 부스팅도 특성 중요도를 제공한다.<br>
랜덤 포레스트보다 당도에 더 집중하는 것을 알 수 있다.

In [24]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


<hr><br>

### **히스토그램 기반 그레이디언트 부스팅**(Histogram-based Gradient Boosting)

그레이디언트 부스팅의 속도를 개선한 것으로 가장 뛰어난 앙상블 학습으로 평가받는 알고리즘이다.<br>


훈련 데이터를 256개의 구간으로 변환하여 사용하기 때문에 노드 분할 속도가 매우 빠르다.<br>
256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해 사용하기 때문에 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.

와인 데이터셋에 HistGradientBoostingClassifier 클래스를 적용해보자.

In [25]:
#from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공한다.<br><br>
이제 특성 중요도를 확인해보자.

히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 `permutation_importance()`를 사용할 것이다.<br>
이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지를 계산한다.

In [28]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input,train_target)
# n_repeats ~> 랜덤하게 섞을 횟수
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance()는 다음과 같은 객체를 반환한다.
- importances: 특성 중요도
- importances_mean: 평균
- importances_std: 표준편차
<br>
평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있다.

이번에는 테스트 세트에서 특성 중요도를 계산해볼 것이다.

In [29]:
 result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
 print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있다는 것을 알 수 있다.<br>
이런 분석을 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상할 수 있다.

그럼 HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인해보자.

In [30]:
hgb.score(test_input, test_target)

0.8723076923076923

약 87%의 정확도로, 앙상블 모델은 확실히 단일 결정 트리보다 좋은 결과를 얻을 수 있다는 것을 알 수 있다.<br>

코랩에는 사이킷런뿐만 아니라 히스토그램 기반 그레이디언트 부스팅 라이브러리인 **XGBoost**와 **LightGBM**이 설치되어 있어 사용해볼 수 있다.<br>
사이킷런의 cross_validate()와 함께 사용할 수 있다.

In [31]:
from xgboost import XGBClassifier

# tree_method='hist' ~> 히스토그램 기반 그레이디언트 부스팅 사용
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [32]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
